In [7]:
import asyncio
import json
from sofascore_wrapper.api import SofascoreAPI
from sofascore_wrapper.search import Search
from sofascore_wrapper.league import League
from sofascore_wrapper.team import Team

api = SofascoreAPI()

# get_flag_team = Team(api, 865207)  
# morocco_flag = await get_flag_team.image()
# morocco_players = await get_flag_team.squad()

# search = Search(api, search_string="Africa Cup of Nations")
# competition = await search.search_all(sport="football")

# can_id = competition['results'][0]['entity']['id']

can_league = League(api, 16)
can_seasons = await can_league.get_seasons()

latest_can_season_id = can_seasons[0].get('id')

can_rounds = await can_league.rounds(latest_can_season_id)

can_round_one = can_rounds['rounds'][0]['round']

rounds_list = [r['round'] for r in can_rounds['rounds']]


# match_can = await can_league.league_fixtures_per_round( latest_can_season_id,1)

# match_can = await can_league.cup_fixtures_per_round( latest_can_season_id, 1)

can_fixtures = await can_league.cup_tree(latest_can_season_id)
standings = await can_league.standings(latest_can_season_id)
from sofascore_wrapper.match import Match
# get_linups = Match(api, 13424932)
# incidents = await get_linups.incidents() 
# team_managers = await get_linups.managers() 
# away_team_linups = await get_linups.lineups_away()
# home_team_linups = await get_linups.lineups_home()
# match_stats = await get_linups.stats()
from sofascore_wrapper.manager import Manager
# get_linups = Match(api, 13424932)
# manager = Manager(api, team_managers.get('homeManager').get('id'))
# team_manager_details = await manager.get_manager()

from sofascore_wrapper.player import Player
get_player = Player(api, 217704) # replace with actual player id
# player_details = await get_player.league_stats(league_id, season_id)

team = Team(api, 865207)
team_stats = await team.league_stats(16, latest_can_season_id)
match_data = Match(api, 13424932)
incidents= Match.incidents()

await api.close()

Exception: Failed to fetch /team/865207/unique-tournament/16/season/58210/statistics/overall: 404

In [41]:
import asyncio
import json
from sofascore_wrapper.api import SofascoreAPI
from sofascore_wrapper.search import Search
from sofascore_wrapper.league import League
from sofascore_wrapper.team import Team
from sofascore_wrapper.match import Match

api = SofascoreAPI()

search_afcon = Search(api, search_string="senegal")
afcon = await search_afcon.search_all(sport="football")
can_id = afcon['results'][0]['entity']['id']
team_senegal = Team(api, can_id)
next_matches = await team_senegal.next_fixtures()
# can_seasons = await can_league.get_seasons()
# last_season = await can_league.current_season()
# last_season_id = last_season.get('id') if last_season else None
# matchs = await can_league.cup_tree(last_afocn_season)
# can_fixtures = await can_league.current_round()
# matchs = Match(api, 15559964)

await api.close()
# last_two_afcon_seasons_ids = [season.get('id') for season in can_seasons[:2]] if can_seasons else []

# await api.close()


In [55]:
from datetime import datetime

def parse_fixture(match: dict) -> dict:
    tournament  = match.get("tournament", {})
    round_info  = match.get("roundInfo", {})
    status      = match.get("status", {})
    home_score  = match.get("homeScore", {})
    away_score  = match.get("awayScore", {})
    time_info   = match.get("time", {})
    timestamp   = match.get("startTimestamp")

    return {
        "sofascore_id":          match["id"],
        "sofascore_league_id":   match["tournament"]["uniqueTournament"]["id"],
        "sofascore_season_id":   match["season"]["id"],
        "sofascore_home_id":     match["homeTeam"]["id"],
        "sofascore_away_id":     match["awayTeam"]["id"],

        "home_team":             match["homeTeam"]["name"],
        "away_team":             match["awayTeam"]["name"],
        "date":                  datetime.fromtimestamp(timestamp) if timestamp else None,
        "timestamp":             timestamp,

        "round":                 round_info.get("round"),
        "round_name":            round_info.get("name"),
        "group_name":            tournament.get("groupName"),
        "group_sign":            tournament.get("groupSign"),

        "status":                status.get("type"),
        "status_long":           status.get("description"),
        "elapsed":               time_info.get("played"),

        "home_score":            home_score.get("current"),
        "away_score":            away_score.get("current"),
        "home_score_period1":    home_score.get("period1"),
        "away_score_period1":    away_score.get("period1"),
        "home_score_period2":    home_score.get("period2"),
        "away_score_period2":    away_score.get("period2"),
        "home_score_normaltime": home_score.get("normaltime"),
        "away_score_normaltime": away_score.get("normaltime"),

        "is_live":               status.get("type") == "inprogress",
        "has_lineups":           False,
        "has_statistics":        False,
        "has_events":            False,
    }

seen = set()
friendly_matches = []
for match in next_matches:
    if match['tournament']['slug'] == 'int-friendly-games':
        key = (match['startTimestamp'], match['homeTeam']['name'], match['awayTeam']['name'])
        if key not in seen:
            seen.add(key)
            friendly_matches.append(match)

# print(json.dumps(friendly_matches, indent=2))
# Sur tes matchs amicaux filtrés
parsed = [parse_fixture(m) for m in friendly_matches]

# Visualiser proprement
import pandas as pd
pd.DataFrame(parsed)

,sofascore_id,sofascore_league_id,sofascore_season_id,sofascore_home_id,sofascore_away_id,home_team,away_team,date,timestamp,round,...,home_score_period1,away_score_period1,home_score_period2,away_score_period2,home_score_normaltime,away_score_normaltime,is_live,has_lineups,has_statistics,has_events
0,15362151,851,87155,4724,4739,USA,Senegal,2026-05-31 19:30:00,1780255800,None,...,None,None,None,None,None,None,False,False,False,False
1,15598097,851,87155,4739,4762,Senegal,Gambia,2026-03-31 19:00:00,1774983600,None,...,None,None,None,None,None,None,False,False,False,False
2,15598098,851,87155,4739,4790,Senegal,Peru,2026-03-28 16:00:00,1774713600,None,...,None,None,None,None,None,None,False,False,False,False


In [54]:
parsed

[{'sofascore_id': 15362151,
  'sofascore_league_id': 851,
  'sofascore_season_id': 87155,
  'sofascore_home_id': 4724,
  'sofascore_away_id': 4739,
  'home_team': 'USA',
  'away_team': 'Senegal',
  'date': datetime.datetime(2026, 5, 31, 19, 30),
  'timestamp': 1780255800,
  'round': None,
  'round_name': None,
  'group_name': None,
  'group_sign': None,
  'status': 'notstarted',
  'status_long': 'Not started',
  'elapsed': None,
  'home_score': None,
  'away_score': None,
  'home_score_period1': None,
  'away_score_period1': None,
  'home_score_period2': None,
  'away_score_period2': None,
  'home_score_normaltime': None,
  'away_score_normaltime': None,
  'is_live': False,
  'has_lineups': False,
  'has_statistics': False,
  'has_events': False},
 {'sofascore_id': 15598097,
  'sofascore_league_id': 851,
  'sofascore_season_id': 87155,
  'sofascore_home_id': 4739,
  'sofascore_away_id': 4762,
  'home_team': 'Senegal',
  'away_team': 'Gambia',
  'date': datetime.datetime(2026, 3, 31, 19

### --------------------------------------------